In [ ]:
!pip install sqlalchemy

In [ ]:
import pymysql
import getpass
import pandas as pd
import sqlalchemy as sa

In [ ]:
password = getpass.getpass('Admin password?')

In [ ]:
password = 'r*X87M5YhEzzRnE'

In [ ]:
host = 'database-1.c9zriyyu6mtc.us-east-1.rds.amazonaws.com'
conn = pymysql.connect(
    host=host,
    user='admin',
    port=3306,
    password=password,
)

In [ ]:
with conn.cursor() as c:
    c.execute('DROP DATABASE class')
    c.execute("DROP USER 'newuser'")

In [ ]:
with conn.cursor() as c:
    c.execute(f'CREATE DATABASE class')
    c.execute(f"CREATE USER 'newuser' IDENTIFIED BY 'password'")
    c.execute(f"GRANT ALL PRIVILEGES ON class . * TO 'newuser'")

In [ ]:
conn = pymysql.connect(
    host=host,
    user='newuser',
    password='password',
    db='class'
)

In [ ]:
transactions = pd.read_csv('./data/Sacramentorealestatetransactions.csv')
transactions.head()

In [ ]:
engine = sa.create_engine(f'mysql+pymysql://newuser:password@{host}/class')
transactions.to_sql('transactions', engine)

In [ ]:
stocks = pd.read_csv('./data/closing-prices.csv', index_col=0)
stocks.head()

In [ ]:
def segment(symbol):
    stocks_values = pd.DataFrame(stocks[symbol])
    stocks_values.index.name = 'date'
    stocks_values['symbol'] = symbol
    stocks_values = stocks_values.reset_index()
    cols = stocks_values.columns
    print(cols)
    stocks_values.columns = 'date price symbol'.split()
    return stocks_values


In [ ]:
stock_table_data = pd.concat([segment(symbol) for symbol in 'F TSLA GOOG IBM AAPL'.split()])
stock_table_data.shape

In [ ]:
stock_table_data.head()

In [ ]:
import sqlite3
with sqlite3.connect('./data/stocks.db') as scon:
    stock_table_data.to_sql('stocks_data', scon, index=False, if_exists='replace')

In [ ]:
states = pd.read_csv('./data/states.csv') # , index_col=['Abbreviation'])
states.head()

In [ ]:
engine = sa.create_engine(f'mysql+pymysql://newuser:password@{host}/class')
states.to_sql('states', engine)